In [ ]:
from redis import Redis
REDIS = Redis('this_redis')

# Redis

One interesting use case is to have Redis track the evolution of some long running iterative process.

In [ ]:
import time

def some_iterative_process():
    time.sleep(1)

In [ ]:
some_iterative_process()

In [ ]:
count = 0
REDIS.set('count', count)

In [ ]:
while count < 30:
    some_iterative_process()
    count = REDIS.incr('count')

In [ ]:
count

## Passing Parameters Between Processes

We could also use Redis to pass parameters between procesess.

In [ ]:
import numpy as np
import json
model_params = {'C':list(np.logspace(-3,3,7)),
                'penalty':'l1',
                'solver' : 'newton-cg'}

In [ ]:
model_params

In [ ]:
type(model_params)

In [ ]:
json.dumps(model_params)

In [ ]:
REDIS.set('model_params', json.dumps(model_params))

## Passing Arrays Between Processes

We could also use Redis to pass arrays between procesess.

In [ ]:
A = np.array([[1,1,1],
              [2,2,2],
              [3,3,3]])

In [ ]:
n, m = A.shape

In [ ]:
encoded_A = A.ravel().tostring()

In [ ]:
REDIS.set('encoded_A', encoded_A)

In [ ]:
REDIS.set('A_n', n)
REDIS.set('A_m', m)

In [ ]:
def matrix_to_redis(matrix, redis_client, key):
    n, m = matrix.shape
    encoded_mat = matrix.ravel().tostring()
    redis_client.set(key, encoded_mat)
    redis_client.set(key+'_n', n)
    redis_client.set(key+'_m', m)

In [ ]:
matrix_to_redis(A, REDIS, 'new_A')

This could also be done via the `pickle` library.

In [ ]:
import pickle
import numpy as np

B = np.random.randint(1,10,9).reshape(3,3)
B_pkl = pickle.dumps(B)

REDIS.set('B', B_pkl)

In [ ]:
B